# Time-bounded Monte Carlo Tree Search for 2048

In this Lab, we will create a time-bounded Monte Carlo Tree Search-powered player for the game of 2048. This is an incredibly challenging & stochastic game that scales in difficulty exponentially as the game continues. Because of this, neural networks aren't particularly great at playing 2048. This is why other algorithms, like Minimax with Alpha-Beta pruning, Expectimax, and Monte Carlo Tree Search are used instead.

You're going to start by importing the only important library for this lab, Foundation:

In [ ]:
import Foundation

OK - now bear with me - I'm going to give you an absolute unit of a code cell. This is all the code that goes behind managing the 2048 game and making it work *FAST*. Now, this is a lot more code than is necessary, but if you want the code to perform as fast as it does now, you need this bitboard implementation.

In [ ]:
extension String {
    func paddedZeros(length: Int) -> String {
        if count == length {
            return self
        }
        return ([Character](repeating: "0", count: length - count) + Array(self)).map({ String($0) }).joined()
    }
}

extension Int {
    func paddedString() -> String {
        var str = Array(String(self == 0 ? 0 : (1 << self))).map({ String($0) })
        str = [String](repeating: " ", count: 4 - str.count) + str
        return str.joined()
    }
}

typealias Board = UInt64
typealias Row = UInt16

let ROW1: UInt64 = 0xFFFF000000000000
let ROW2: UInt64 = 0xFFFF00000000
let ROW3: UInt64 = 0xFFFF0000
let ROW4: UInt64 = 0xFFFF

struct Utils2048 {
    var rowLeftTable: [Row: Row]
    var rowRightTable: [Row: Row]
    var scoreTable: [Row: Double]
    var randomTileTable: [Row: [Row]]

    static var shared = Utils2048()

    init() {
        rowLeftTable = [:]
        rowRightTable = [:]
        scoreTable = [:]
        randomTileTable = [:]
        initializeTables()
    }

    mutating func initializeTables() {
        print("Initializing tables...")
        for row in 0..<UInt16.max {
            var line = [(row & 0xF000) >> 12,
                        (row & 0xF00) >> 8,
                        (row & 0xF0) >> 4,
                        (row & 0xF) >> 0]

            var newLines: [[UInt16]] = []
            for i in 0..<4 {
                if line[i] == 0 {
                    for _ in 1...9 {
                        newLines.append(line)
                        newLines[newLines.count - 1][i] = 1
                    }
                    newLines.append(line)
                    newLines[newLines.count - 1][i] = 2
                }
            }
            randomTileTable[row] = newLines.map({ Utils2048.row(line: $0) })

            let scoreValues = line.map { v -> UInt in
                let v = UInt(v)
                return v == 0 ? 0 : ((1 << v) * (v - 1))
            }
            scoreTable[row] = Double(scoreValues.reduce(0, +))

            var newLine: [UInt16] = [0, 0, 0, 0]
            var j = 0
            var previous: UInt16? = nil
            for i in 0..<4 {
                if line[i] != 0 {
                    if previous == nil {
                        previous = line[i]
                    } else {
                        if previous == line[i] {
                            newLine[j] = line[i] + 1
                            j += 1
                            previous = nil
                        } else {
                            newLine[j] = previous!
                            j += 1
                            previous = line[i]
                        }
                    }
                }
            }
            if previous != nil {
                newLine[j] = previous!
            }
            line = newLine

            var row = row
            var result = (line[0] << 12) | (line[1] << 8) | (line[2] << 4) | (line[3] << 0)

            rowLeftTable[row] = result
            Utils2048.reverse(row: &result)
            Utils2048.reverse(row: &row)
            rowRightTable[row] = result
        }
    }

    static func reverse(row: inout Row) {
        row = (row >> 12) | ((row >> 4) & 0x00F0) | ((row << 4) & 0x0F00) | (row << 12)
    }

    static func line(row: Row) -> [Row] {
        let line = [(row & 0xF000) >> 12,
                    (row & 0xF00) >> 8,
                    (row & 0xF0) >> 4,
                    (row & 0xF) >> 0]
        return line
    }

    static func row(line: [UInt16]) -> Row {
        return (line[0] << 12) | (line[1] << 8) | (line[2] << 4) | (line[3] << 0)
    }

    static func rows(board: Board) -> [Row] {
        let rows = [
            Row(board >> 48),
            Row((board >> 32) & 0xFFFF),
            Row((board >> 16) & 0xFFFF),
            Row((board >>  0) & 0xFFFF)
        ]
        return rows
    }

    static func lines(board: Board) -> [[Row]] {
        let lines = rows(board: board).map({ line(row: $0) })
        return lines
    }

    static func board(rows: [Row]) -> Board {
        let rows = rows.map({ UInt64($0) })
        return (rows[0] << 48) | (rows[1] << 32) | (rows[2] << 16) | (rows[3] << 0)
    }

    static func transpose(board x: inout Board) {
        let a1 = x & 0xF0F00F0FF0F00F0F
        let a2 = x & 0x0000F0F00000F0F0
        let a3 = x & 0x0F0F00000F0F0000
        let a = a1 | (a2 << 12) | (a3 >> 12)
        let b1 = a & 0xFF00FF0000FF00FF
        let b2 = a & 0x00FF00FF00000000
        let b3 = a & 0x00000000FF00FF00
        x = b1 | (b2 >> 24) | (b3 << 24)
    }

    static func emptyTiles(board x: Board) -> Int {
        guard x != 0 else {
            return 16
        }
        var x = x
        x |= (x >> 2) & 0x3333333333333333
        x |= (x >> 1)
        x = ~x & 0x1111111111111111
        x += x >> 32
        x += x >> 16
        x += x >>  8
        x += x >>  4
        return Int(x & 0xf)
    }

    static func insertRandomTile(board: Board) -> Board {
        var boardRows = rows(board: board)
        let randomRowChange = boardRows.enumerated()
                                       .map({ ($0.offset, Utils2048.shared.randomTileTable[$0.element]!) })
                                       .filter({ !$0.1.isEmpty }).randomElement()!
        boardRows[randomRowChange.0] = randomRowChange.1.randomElement()!
        return Utils2048.board(rows: boardRows)
    }

    static func printBoard(board: Board) {
        let board = lines(board: board)
        for r in 0..<4 {
            for c in 0..<4 {
                let powerVal = board[r][c]
                print(powerVal == 0 ? 0 : (1 << powerVal), terminator: "|")
            }
            print("")
        }
        print("")
    }
}

struct Game2048 {
    var board: Board

    var score: Double {
        let baseScore = Utils2048.rows(board: board).map({ Utils2048.shared.scoreTable[$0]! }).reduce(0, +)
        return baseScore
    }

    var validDirections: [Direction] {
        var dirs: [Direction] = []
        if moved(direction: .up).board != board { dirs.append(.up) }
        if moved(direction: .down).board != board { dirs.append(.down) }
        if moved(direction: .left).board != board { dirs.append(.left) }
        if moved(direction: .right).board != board { dirs.append(.right) }
        return dirs
    }

    var gameWon: Bool {
        return score >= (2048 * 10)
    }

    var gameOver: Bool {
        return gameWon || (Utils2048.emptyTiles(board: board) == 0 && validDirections.count == 0)
    }

    enum Direction {
        case up, down, left, right

        var string: String {
            switch self {
            case .up:
                return "up"
            case .down:
                return "down"
            case .left:
                return "left"
            case .right:
                return "right"
            }
        }
    }

    private mutating func moveHorizontal(lookup: [Row: Row]) {
        let row1 = Board(lookup[Row((board & ROW1) >> 48)]!)
        let row2 = Board(lookup[Row((board & ROW2) >> 32)]!)
        let row3 = Board(lookup[Row((board & ROW3) >> 16)]!)
        let row4 = Board(lookup[Row((board & ROW4) >>  0)]!)
        board = (row1 << 48) | (row2 << 32) | (row3 << 16) | (row4 << 0)
    }

    private mutating func moveVertical(lookup: [Row: Row]) {
        Utils2048.transpose(board: &board)
        moveHorizontal(lookup: lookup)
        Utils2048.transpose(board: &board)
    }

    mutating func moveLeft() {
        moveHorizontal(lookup: Utils2048.shared.rowLeftTable)
    }

    mutating func moveRight() {
        moveHorizontal(lookup: Utils2048.shared.rowRightTable)
    }

    mutating func moveUp() {
        moveVertical(lookup: Utils2048.shared.rowLeftTable)
    }

    mutating func moveDown() {
        moveVertical(lookup: Utils2048.shared.rowRightTable)
    }

    mutating func move(direction: Direction) {
        switch direction {
        case .up:
            moveUp()
        case .down:
            moveDown()
        case .left:
            moveLeft()
        case .right:
            moveRight()
        }
    }

    func moved(direction: Direction) -> Game2048 {
        var new = copy()
        new.move(direction: direction)
        return new
    }

    mutating func play(direction: Direction) {
        move(direction: direction)
        board = Utils2048.insertRandomTile(board: board)
    }

    func print() {
        let board = Utils2048.lines(board: self.board).map({ $0.map({ Int($0) }) })
        var temp = ""
        temp += [String](repeating: "-", count: "|    |    |    |    |".count).joined()
        temp += "\n"
        temp += "|\(board[0][0].paddedString())|\(board[0][1].paddedString())|\(board[0][2].paddedString())|\(board[0][3].paddedString())|"
        temp += "\n"
        temp += [String](repeating: "-", count: "|    |    |    |    |".count).joined()
        temp += "\n"
        temp += "|\(board[1][0].paddedString())|\(board[1][1].paddedString())|\(board[1][2].paddedString())|\(board[1][3].paddedString())|"
        temp += "\n"
        temp += [String](repeating: "-", count: "|    |    |    |    |".count).joined()
        temp += "\n"
        temp += "|\(board[2][0].paddedString())|\(board[2][1].paddedString())|\(board[2][2].paddedString())|\(board[2][3].paddedString())|"
        temp += "\n"
        temp += [String](repeating: "-", count: "|    |    |    |    |".count).joined()
        temp += "\n"
        temp += "|\(board[3][0].paddedString())|\(board[3][1].paddedString())|\(board[3][2].paddedString())|\(board[3][3].paddedString())|"
        temp += "\n"
        temp += [String](repeating: "-", count: "|    |    |    |    |".count).joined()
        temp += "\n"
        Swift.print(temp)
    }

    func copy() -> Game2048 {
        return Game2048(board: board)
    }

    func possibleNextSteps() -> [Game2048] {
        var nextSteps: [Game2048] = []
        var push = 0
        for _ in 1...16 {
            if (board >> push) & 0xf == 0 {
                nextSteps.append(Game2048(board: board | Board([1, 1, 1, 1, 1, 1, 1, 1, 1, 2].randomElement()! << push)))
            }
            push += 4
        }
        return nextSteps
    }
}

extension Game2048 {
    init() {
        board = Utils2048.insertRandomTile(board: Utils2048.insertRandomTile(board: 0))
    }

    init(board: [[Int]]) {
        let str = board.flatMap({ $0 }).map({ $0 == 0 ? "0000" : String(Int(log(Float($0)) / log(2)), radix: 2).paddedZeros(length: 4) }).joined()
        self.board = Board(str, radix: 2)!
    }
}

<sup>*in awe at the size of this cell. absolute unit*</sup>

Anyway, now that we've got that out of the way, we can focus on what you're here to learn: how do you play this thing automatically?

At the heart of this method, it's quite simple: but results in a very efficient player! In order to play random moves in a time-bounded environment in parallel, let's create a function `monteCarloSearch(state: Game2048) -> Game2048.Direction`. Essentially, this function will take a game state, and will return a direction in which to move - the direction that MCTS believes is currently optimal.

Inside of this function, there's a nested helper function - `performRollout(state: Game2048, direction: Game2048.Direction) -> Double`. This function will take a game state and a direction to score, and will perform random moves after the initial one. Once it finishes the game (either loses or wins), it'll return the score of the board.

Then, the meat of the algorithm: you start by noting down the valid directions (which moves are currently legal). If there's only one legal move, then it's the only move that you can take regardless of future score - so return it. If not, then continue with the algorithm.

Next, you define the maximum time that the algorithm is allowed to take to make a decision. In this case, I've set it to 0.015 seconds. If you have slower performing hardware, you can set it to a higher number to get a better player. Remember, the more time you allocate to MCTS, the better! At this time, you'll also initialize the scores for all 4 possible directions - initially, these are zero. At least one move must be legal at this point, and therefore, at least one move will have a score higher than zero when the algorithm is done executing.

Now it's time to actually score the moves! If the machine you're running on has more than 1 core, it'll distribute this process on as many as possible, up until a maximum of 4 cores (as in, one direction being scored per core). In order to make this easy, Apple's cross-platform & Swift-compatible `Dispatch` library has a static function `concurrentPerform`, which enables you to perform many iterations of a task concurrently. In this case, the number of iterations is equal to the number of possible directions to move in. `concurrentPerform` is fed with an anonymous function (also called a [closure](https://docs.swift.org/swift-book/LanguageGuide/Closures.html)). This closure takes the current iteration number, and does some processing.

In this case, the "processing" refers to the scoring of a direction. This is how the scoring is done:

1. Create an array of Double values called `scores`, to store the scores that are calculated.
1. Create a new integer variable `rollouts`, initialized to zero, to keep track of the number of rollouts that are run.
1. Define a constant that contains the maximum time until which the rollouts are allowed to be calculated.
1. Create a repeat-while loop, running the following steps:
    1. Increment `rollouts`.
    1. Perform a rollout and store the score.
    1. Append the score to `scores`.
    1. Repeat if the current time is less than or equal to the maximum time alloted
1. Determine the average score by dividing the sum of `scores` by `rollouts`.
1. Set the respective direction's score.

Finally, put the directions & their scores into a dictionary, and return the direction with the highest average score.

In [ ]:
func monteCarloSearch(state: Game2048) -> Game2048.Direction {
    func performRollout(state: Game2048, direction: Game2048.Direction) -> Double {
        var newState = state.copy() // Copy the initial state.
        newState.play(direction: direction) // Move in the direction to be scored.
        while !newState.gameOver { // Repeat until game is over.
            newState.play(direction: newState.validDirections.randomElement()!) // Make a random (valid) move.
        }
        return newState.score // Return the score of the game.
    }

    let directions = state.validDirections // Determine valid (legal) moves.

    if directions.count == 1 { // If there's only one move left ...
        return directions[0] // ... return that move.
    }

    let maxTime = 0.015 // Duration above which no more rollouts may occur.
    // Scores for directions.
    var scoreUp = 0.0
    var scoreDown = 0.0
    var scoreLeft = 0.0
    var scoreRight = 0.0

    DispatchQueue.concurrentPerform(iterations: directions.count) { dirIndex in // Perform each direction's calculations in parallel.
        let direction = directions[dirIndex] // Get the relevant direction.
        var scores: [Double] = [] // Declare `scores` - the scores of the evaluated rollouts.
        var rollouts = 0 // Declare `rollouts` - the number of rollouts evaluated.
        let timeBarrier = DispatchTime.now() + maxTime // Time ceiling above which no more rollouts may occur.
        repeat { // Repeat ...
            rollouts += 1 // ... increment `rollouts` ...
            let score = performRollout(state: state, direction: direction) // ... perform a rollout ...
            scores.append(score) // ... append the score to `scores`.
        } while DispatchTime.now() <= timeBarrier // until the current time is above the ceiling.
        let avgScore = scores.reduce(0, +) / Double(rollouts) // Determine average score.
        // Store the score with the respective direction.
        switch direction {
        case .up:
            scoreUp = avgScore
        case .down:
            scoreDown = avgScore
        case .left:
            scoreLeft = avgScore
        case .right:
            scoreRight = avgScore
        }
    }

    let scoreSums: [Game2048.Direction: Double] = [
        .up: scoreUp,
        .down: scoreDown,
        .left: scoreLeft,
        .right: scoreRight
    ]
    return scoreSums.max(by: { a, b in a.value < b.value })!.key // Return direction with highest score.
}

Now that you've got a working 2048 bot, it's time to actually see it in action. The code for this is simple - initialize a new game instance, and run a while loop that will repeat until the game is over. In the loop, determine the best move, play it, and print the new board state.

In [ ]:
var game = Game2048()
let startTime = Date()
var movesMade = 0
while !game.gameOver {
    game.play(direction: monteCarloSearch(state: game))
    movesMade += 1
}
let elapsedTime = Date().timeIntervalSince(startTime)
print("Time elapsed: \(elapsedTime)")
print("Moves made: \(movesMade)")
print("Moves per second: \(Double(movesMade) / elapsedTime)")

As you can see, we're running at around *60 moves per second*! That's amazing for 2048 - it can be played reliably in just ~16 seconds with the power of Time-bounded Monte Carlo Tree Search and the Swift language. Remember, this notebook isn't even running optimized code. When optimizations are turned on, Swift performs orders of magnitude faster.

Now, let's see how you can take this power, and host it on the IBM Cloud with Kitura.

<hr>
<h2>The last exercise!</h2>
<p>Congratulations, you have completed your first lesson and hands-on lab in Python. However, there is one more thing you need to do. The Data Science community encourages sharing work. The best way to share and showcase your work is to share it on GitHub. By sharing your notebook on GitHub you are not only building your reputation with fellow data scientists, but you can also show it off when applying for a job. Even though this was your first piece of work, it is never too early to start building good habits. So, please read and follow <a href=“https://cognitiveclass.ai/blog/data-scientists-stand-out-by-sharing-your-notebooks/” target=“_blank”>this article</a> to learn how to share your work.
<hr>